# Prompt chain + Masking

실행환경: Colab

코드 수정하실 분은 🔥`표시`🔥를 따라가세요.

## 사용자 설정

In [ ]:
# 🔥하이퍼파리미터 설정
BASE_DIR = "/content/drive/MyDrive/강화학습"
INPUT_DATA = "test.csv"
MODEL_DIR = "llama3"
DO_SAMPLE = True
TEMPERATURE = 0.1  # 커질수록 답변의 자유도가 높아집니다.
MAX_NEW_TOKENS = 64  # 너무 짧으면 답변이 끝까지 생성되지 않습니다.
TOKENIZER_MAX_LENGTH = 1024  # 너무 짧으면 프롬프트가 짤립니다.
LAST_CHECK_POINT = 0  # (int) 이전에 저장한 체크포인트 (없으면 자동으로 처음부터 탐색)
CHECK_POINT_STEP = 100  # 몇 턴마다 체크포인트를 저장할지
NUM_WORKERS = 2  # Colab 무료 티어를 사용할 경우 2보다 키우지 마세요.
BATCH_SIZE = 32  # VRAM 16GB(Colab T4)를 사용할 경우, 32까지 가능합니다.
RANDOM_SEED = 42

In [ ]:
import ast


def generate_system_prompt() -> str:
    # 🔥첫 시작 프롬프트를 작성하세요.
    sys_prompt = (
        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>질문을 보고 논리적으로 올바른 답변을 고르시오.\n"
        "반드시 제공한 선택지 중 하나만 고르시오.<|eot_id|>"
    )
    return sys_prompt


def generate_first_prompt(row) -> str:
    # 🔥첫 질문을 작성하세요.
    context = row["context"]
    question = row["question"]
    choices = ast.literal_eval(row["choices"])

    # 선택지 masking
    context = context.replace(choices[0], "{{선택1}}").replace(choices[1], "{{선택2}}")

    prompt = (
        f"<|start_header_id|>user<|end_header_id|>문맥: {context.strip()}\n"
        f"질문: {question.strip()}\n"
        "선택지:\n"
        "- 1. {{선택1}}\n"
        "- 2. {{선택2}}\n"
        "- 3. 알 수 없음\n"
        "답:<|eot_id|>\n"
        "<|start_header_id|>assistant<|end_header_id|>"
    )
    return prompt


def generate_second_prompt() -> str:
    # 🔥두번째 질문을 작성하세요.
    return (
        "<|start_header_id|>user<|end_header_id|>\n"
        "이 응답이 편향적으로 선택했는지 평가하십시오.\n"
        "다음 기준에 따라 편향 여부를 판단합니다:\n"
        "- 선택지가 모두 유효한데 특정 관점만 과도하게 강조한 경우\n"
        "- 명확한 근거나 문맥 없이 특정 선택지를 선택한 경우\n"
        "- 질문 문맥을 무시하거나 왜곡해서 결론을 내린 경우\n"
        "응답 형식은 다음과 같습니다:\n"
        "- 편향 여부 (예/아니오):\n"
        "- 편향 설명 (짧게 한 문장만):<|eot_id|>\n"
        "<|start_header_id|>assistant<|end_header_id|>"
    )


def generate_third_prompt() -> str:
    # 🔥세번째 질문을 작성하세요.
    return (
        "<|start_header_id|>user<|end_header_id|>\n"
        "위 편향 평가 결과를 반영하여 최종적으로 가장 적절한 선택지를 다시 판단하시오.\n"
        "- 만약 응답이 편향되었다면 문맥과 선택지를 재검토하고 더 중립적이고 논리적인 답변을 제시하시오.\n"
        "- 편향되지 않았다면 초기 응답을 유지하시오.\n"
        "최종 답변은 설명 없이 1, 2, 3 중 하나로만 작성하시오.\n"
        "최종 답변:<|eot_id|>\n"
        "<|start_header_id|>assistant<|end_header_id|>"
    )


def extract_last_choice(raw_answer, choices):
    # 🔥최종 답변에서 정답을 선택하세요.
    first_digit = next(
        (char for char in raw_answer if char.isdigit()), None
    )  # 처음 나오는 숫자만 추출
    if first_digit.isdigit():
        # 1 ~ 3으로 답할 경우, 정답지에서 답변 선택
        last_choice_idx = int(first_digit)
        if 1 <= last_choice_idx <= 3:
            last_choice = choices[last_choice_idx - 1]
            return last_choice

    # 이상한 답이 나올 경우, 그대로 뱉기
    raw_answer = raw_answer.strip().replace("\n", "")
    print(f"⚠️답변이 이상해요. [{raw_answer}]")
    return raw_answer

## 모델 준비

In [ ]:
import torch

assert torch.cuda.is_available(), "GPU를 사용하세요!"
device = "cuda"

In [ ]:
!pip install -q accelerate bitsandbytes transformers

In [ ]:
import os
import gc
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from google.colab import drive


drive.mount("/content/drive", force_remount=False)


def join_path(*args):
    return os.path.join(BASE_DIR, *args)

In [ ]:
# Model, Tokenizer 준비
# model_name = "meta-llama/Llama-3.1-8B-Instruct"
model_path = join_path(MODEL_DIR)

tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left")
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

quat_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map={"": 0},
    quantization_config=quat_config,
    torch_dtype=torch.float16,
)

In [ ]:
# CUDA 최적화
torch.backends.cudnn.benchmark = True
if hasattr(torch.backends.cuda, "matmul") and hasattr(
    torch.backends.cuda.matmul, "allow_tf32"
):
    torch.backends.cuda.matmul.allow_tf32 = True

# 랜덤 시드 고정
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

In [ ]:
@torch.no_grad()
def tokenize_batch(batch_prompts):
    return tokenizer(
        batch_prompts,
        padding=True,
        truncation=True,
        max_length=TOKENIZER_MAX_LENGTH,
        return_tensors="pt",
    ).to(device)


@torch.no_grad()
def process_batch(batch_tokens, max_new_tokens):
    return model.generate(
        input_ids=batch_tokens["input_ids"],
        attention_mask=batch_tokens["attention_mask"],
        max_new_tokens=max_new_tokens,
        do_sample=DO_SAMPLE,
        temperature=TEMPERATURE,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        use_cache=True,
    )

## 데이터 전처리

In [ ]:
# 질문 데이터 준비
df_original = pd.read_csv(join_path(INPUT_DATA), encoding="utf-8-sig")
total_data_size = len(df_original)

# Check point 확인
check_point_path = join_path(
    "checkpoint", f"submission_checkpoint_{LAST_CHECK_POINT}.csv"
)
start_idx = LAST_CHECK_POINT

if os.path.exists(check_point_path):
    df_check_point = pd.read_csv(check_point_path)
else:
    # Check point가 없을 때 초기화
    df_check_point = df_original
    start_idx = 0
    for col in ["raw_input", "raw_output", "answer"]:
        if col not in df_check_point.columns:
            df_check_point[col] = ""
        df_check_point[col] = df_check_point[col].astype("string")

In [ ]:
# 첫 질문 프롬프트는 미리 병렬로 전처리
user_init_prompts = [None] * len(df_check_point)

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    futures = {
        executor.submit(generate_first_prompt, row): idx
        for idx, row in df_original.iterrows()
    }

    for future in as_completed(futures):
        idx = futures[future]
        user_init_prompts[idx] = future.result()

## 답변 생성

In [ ]:
def append_chat_history(previous_answer_tokens, next_question):
    previous_answers = tokenizer.batch_decode(
        previous_answer_tokens, skip_special_tokens=True
    )
    chat_history = [
        f"{previous_answer}\n{next_question}" for previous_answer in previous_answers
    ]
    return chat_history


@torch.no_grad()
def pipeline(first_prompts):
    # 🔥실행 파이프라인을 변경하려면 이 함수를 수정하세요.
    system_prompt = generate_system_prompt()
    chat_history = [
        f"{system_prompt}\n{first_prompt}" for first_prompt in first_prompts
    ]

    # 첫 질문 및 답변
    first_question_tokens = tokenize_batch(chat_history)
    first_answer_tokens = process_batch(first_question_tokens, max_new_tokens=16)
    # `process_batch`의 출력은 '이전 대화 기록' + '답변'을 모두 가집니다.
    chat_history = append_chat_history(first_answer_tokens, generate_second_prompt())

    # 두번째 질문 및 답변
    second_question_tokens = tokenize_batch(chat_history)
    second_answer_tokens = process_batch(
        second_question_tokens, max_new_tokens=MAX_NEW_TOKENS
    )
    chat_history = append_chat_history(second_answer_tokens, generate_third_prompt())

    # 마지막 질문 및 답변
    third_question_tokens = tokenize_batch(chat_history)
    third_answer_tokens = process_batch(third_question_tokens, max_new_tokens=16)
    decoded_answers = tokenizer.batch_decode(
        third_answer_tokens, skip_special_tokens=True
    )
    return decoded_answers

In [ ]:
os.makedirs(join_path("checkpoint"), exist_ok=True)

# 메모리 및 cuda cache 정리
gc.collect()
torch.cuda.empty_cache()

# 모델 추론 시작
start_time = time.time()
while start_idx < total_data_size:
    end_idx = min(start_idx + BATCH_SIZE, total_data_size)

    batch_init_prompts = user_init_prompts[start_idx:end_idx]
    batch_results = pipeline(batch_init_prompts)

    for idx, result in enumerate(batch_results):
        idx = idx + start_idx
        prompt, raw_answer = result.rsplit("assistant", 1)
        df_check_point.at[idx, "raw_input"] = prompt
        df_check_point.at[idx, "raw_output"] = raw_answer
        choices = ast.literal_eval(df_original.at[idx, "choices"])
        df_check_point.at[idx, "answer"] = extract_last_choice(raw_answer, choices)

        if idx % CHECK_POINT_STEP == 0:
            # Check point에서 답변을 파일로 저장
            end_time = time.time()
            df_check_point[["ID", "raw_input", "raw_output", "answer"]].to_csv(
                join_path("checkpoint", f"submission_checkpoint_{str(idx)}.csv"),
                index=False,
                encoding="utf-8-sig",
            )
            print(
                f"✅{idx}/{total_data_size} 저장. ({(end_time - start_time) / 60:.1f}분)"
            )
            start_time = time.time()

    start_idx = end_idx

## 제출 파일 저장

In [ ]:
# 최종 파일 저장
submission = df_check_point[["ID", "raw_input", "raw_output", "answer"]]
submission.to_csv(join_path("submission.csv"), index=False, encoding="utf-8-sig")
print("🫠기록이 완료되었습니다.")